In [3]:
import requests
import pandas as pd
from random_word import Wordnik
import time
import math

In [4]:
def generate_search_strings(num_items: int):    
    batch_size = 10
    num_calls = math.ceil(num_items/batch_size)
    output = []
    wordnik_service = Wordnik()
    for i in range(num_calls):
        try:
            # Return a single random word
            res = wordnik_service.get_random_words(includePartOfSpeech ="noun,verb,adverb",hasDictionaryDef=True, limit=batch_size)
            #TODO: Check if adding a duplicate search term
            [output.append(x) for x in res]
        except Exception as e:
            print(e)
    print('Generated ', int(batch_size * num_calls), ' search terms.' )
    return output

def lexica_search(terms: list):
    search_base='https://lexica.art/api/v1/search?q='
    prompts = pd.DataFrame(columns=['search_string','source','prompt'])
    counter = 0
    for item in terms:
        #print('Searching term: ' , item)
        query = item
        query = query.replace(' ', '+')
        try:
            d = requests.get(url=(search_base + query))
            data = d.json()
            obj = data['images']
        except Exception as e:
            print(e)
            return prompts
        #print('Adding items to db for search term: ', item)
        for item in obj:
            row = [query, item['src'], item['prompt']]
            prompts.loc[item['id']] = row
        counter +=1
        print('Commited prompts for term ', counter, ' out of ', len(terms))
        time.sleep(10)

    return prompts



In [34]:
#main
terms = generate_search_strings(num_items=500)
res = lexica_search(terms=terms)
res.to_json('./seed-prompts-take-5.json', orient='split')

'NoneType' object is not iterable
Generated  500  search terms.
Commited prompts for term  1  out of  490
Commited prompts for term  2  out of  490
Commited prompts for term  3  out of  490
Commited prompts for term  4  out of  490
Commited prompts for term  5  out of  490
Commited prompts for term  6  out of  490
Commited prompts for term  7  out of  490
Commited prompts for term  8  out of  490
Commited prompts for term  9  out of  490
Commited prompts for term  10  out of  490
Commited prompts for term  11  out of  490
Commited prompts for term  12  out of  490
Commited prompts for term  13  out of  490
Commited prompts for term  14  out of  490
Commited prompts for term  15  out of  490
Commited prompts for term  16  out of  490
Commited prompts for term  17  out of  490
Commited prompts for term  18  out of  490
Commited prompts for term  19  out of  490
Commited prompts for term  20  out of  490
Commited prompts for term  21  out of  490
Commited prompts for term  22  out of  490

In [54]:
search_base='https://lexica.art/api/v1/search?q='
query='in the style of'
query = query.replace(' ', '+')
d = requests.get(url=(search_base + query))
data = d.json()
obj = data['images']
images = pd.DataFrame(columns=['source','prompt'])
for item in obj:
    row = [item['src'],item['prompt']]
    images.loc[item['id']] = row


(50, 2)

In [5]:
with open('./1000-most-common.txt') as file:
    lines = file.readlines()
    common = [line.rstrip() for line in lines]
res_common = lexica_search(terms = common)
res_common.to_json('./prompts-with-common.json', orient='split')


In [33]:
wordnik_service = Wordnik()
res = wordnik_service.get_random_words(includePartOfSpeech ="noun,verb,adverb",hasDictionaryDef=True, limit=50)

type(res)

list